In [6]:
## TODO: Write your student number here as a string, for example, student_number = "112233"
student_number = 452056 # write your code here

## Pooling problem 
using JuMP         # JuMP: Modeling language and solver interface
using Ipopt        # Nonlinear programming solver
using CSV          # For reading CSV files
using DataFrames   # For arranging data to a nice format

## Struct for arcs 
struct Arc
    i::String      # Head node of arc (i,j) ∈ A
    j::String      # Tail node of arc (i,j) ∈ A
end

## Struct for nodes 
struct Node
    q1lb::Float64  # Lower bound of property 1
    q1ub::Float64  # Upper bound of property 1
    q2lb::Float64  # Lower bound of property 2
    q2ub::Float64  # Upper bound of property 2
    blb::Float64   # Lower flow bound
    bub::Float64   # Upper flow bound
end

## TODO: Change the name fXXXXXX of this function to match your student number. For example: f112233()
function f452056()

    ## Read arc data
    adata = CSV.read("arcs.csv")

    ## Set arc array
    inode = adata[!, 1]
    jnode = adata[!, 2]
    A     = Arc.(inode, jnode)

    ## Read the node data 
    ndata = CSV.read("nodes.csv")
    nnode = size(ndata, 1)   

    ## Split V into source, pool, and target nodes 
    V     = ndata[!, 1]                                  # All nodes                   
    S     = [V[i] for i = 1:nnode if ndata[i,2] == "s"]  # Source nodes
    T     = [V[i] for i = 1:nnode if ndata[i,2] == "t"]  # Target nodes

    ## Resource bounds of each node
    q1lb  = ndata[!, 3]    # Lower bounds of property 1
    q1ub  = ndata[!, 4]    # Upper bounds of property 1
    q2lb  = ndata[!, 5]    # Lower bounds of property 2
    q2ub  = ndata[!, 6]    # Upper bounds of property 2
    blb   = ndata[!, 7]    # Lower flow bounds
    bub   = ndata[!, 8]    # Upper flow bounds

    ## Set node array and make a dictionary for convenience  
    N = Node.(q1lb, q1ub, q2lb, q2ub, blb, bub)
    N = Dict(V[i] => N[i] for i = 1:size(V,1))

    ## Define model using Nonlinear solver Ipopt
    model = Model(with_optimizer(Ipopt.Optimizer))

    ## Variables
    @variable(model, x[A] >= 0)        # Arc flows
    @variable(model, q1[i in V] >= 0)  # Property 1 values
    @variable(model, q2[i in V] >= 0)  # Property 2 values

       
    ## TODO: Write the objective function. You can look at Excercise 1.5 solution for guidance. 
    N_minus(i) = [j for j in V if Arc(j, i) ∈ A]
    N_plus(i) = [j for j in V if Arc(i, j) ∈ A]
    # (1)
    @variable(model, c[T])
    @constraint(model, c["t1"] == 100 * (2 - q1["t1"] / N["t1"].q1ub))
    @constraint(model, c["t2"] == 150 * (2 - q1["t2"] / N["t2"].q1ub))
    # (2)
    @objective(model, Max, sum(c[t] * sum(x[Arc(j, t)] for j in N_minus(t)) for t in T))

    ## TODO: Define constraints of the model. You can look at Excercise 1.5 solution for guidance. 
    # (3, 4)
    @constraint(model, [s = S], N[s].blb ≤ sum(x[Arc(s, j)] for j in N_plus(s)) ≤ N[s].bub)
    # (5)
    # @constraint(model, [p = P], sum(x[Arc(j, p)] for j in N_minus(p)) == sum(x[Arc(p, j)] for j in N_plus(p)))
    # (6)
    # @constraint(model, [p = P], sum(x[Arc(j, p)] for j in N_minus(p)) ≤ N[p].bub)
    # (7, 8)
    @constraint(model, [t = T], N[t].blb ≤ sum(x[Arc(j, t)] for j in N_minus(t)) ≤ N[t].bub)
    # (9)
    # @constraint(model, [p = P], sum(q1[j] * x[Arc(j, p)] for j in N_minus(p)) == q1[p] * sum(x[Arc(p, j)] for j in N_plus(p)))
    # @constraint(model, [p = P], sum(q2[j] * x[Arc(j, p)] for j in N_minus(p)) == q2[p] * sum(x[Arc(p, j)] for j in N_plus(p)))
    # (10)
    @constraint(model, [t = T], sum(q1[j] * x[Arc(j, t)] for j in N_minus(t)) == q1[t] * sum(x[Arc(j, t)] for j in N_minus(t)))
    @constraint(model, [t = T], sum(q2[j] * x[Arc(j, t)] for j in N_minus(t)) == q2[t] * sum(x[Arc(j, t)] for j in N_minus(t)))
    # (11, 12)
    @constraint(model, [t = T], N[t].q1lb ≤ q1[t] ≤ N[t].q1ub)
    @constraint(model, [t = T], N[t].q2lb ≤ q2[t] ≤ N[t].q2ub)
    # (13)
    @constraint(model, [s = S], q1[s] == N[s].q1lb)  # q1lb = q1ub
    @constraint(model, [s = S], q2[s] == N[s].q2lb)  # q2lb = q2ub
    

    ## TODO: Try different initial (starting) values for pool nodes. Try to find the 2 solutions reported at the end.
    set_start_value(q1["t1"], 1.2)
    set_start_value(q2["t1"], 3.0)
    set_start_value(q1["t2"], 33.0)
    set_start_value(q2["t2"], 38.0)

    # Print the model at any point to see how it looks
    # println(model)

    ## Solve the model
    optimize!(model)
    status = termination_status(model)
    println(status)

    ## Get objecive + solution
    obj = objective_value(model)
    x   = value.(x)
    q1  = value.(q1)
    q2  = value.(q2)

    ## Print the solution and its cost
    println("\nSolution cost: ", round(obj, digits = 4))

    println("\nFlows: \n")
    for a in A
        println("x($(a.i),$(a.j)) = ", round(x[a], digits = 4))
    end

    println("\nProperty 1 values:\n")
    for t in T
        println("q1[$t] = ", round(q1[t], digits = 4))
    end

    println("\nProperty 2 values:\n")
    for t in T
        println("q2[$t] = ", round(q2[t], digits = 4))
    end
    return (x, q1, q2, obj)
end


## TODO: Change the name fXXXXXX() of this function call to match your student number, e.g., f112233().  
(x, q1, q2, obj) = f452056();


## Here is the cost of two solutions that you should be able to find:
##
## Solution cost = 5234.4436

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      292
Number of nonzeros in inequality constraint Jacobian.:      136
Number of nonzeros in Lagrangian Hessian.............:      160

Total number of variables............................:       70
                     variables with only lower bounds:       68
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       38
Total number of inequality constraints...............:       44
        inequality constraints with only lower bounds:       22
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       22

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 


Solution cost: 5234.4436

Flows: 

x(s1,t1) = 0.0
x(s2,t1) = 0.0
x(s3,t1) = 0.0
x(s4,t1) = 0.0
x(s5,t1) = 0.0
x(s6,t1) = 0.0
x(s7,t1) = 0.0
x(s8,t1) = 0.0
x(s9,t1) = 3.0
x(s10,t1) = 3.0
x(s11,t1) = 3.0
x(s12,t1) = 0.9633
x(s13,t1) = 1.0367
x(s14,t1) = 0.0
x(s15,t1) = 0.0
x(s16,t1) = 0.0
x(s1,t2) = 3.0
x(s2,t2) = 0.0
x(s3,t2) = 0.0
x(s4,t2) = 0.0
x(s5,t2) = 3.0
x(s6,t2) = 0.0385
x(s7,t2) = 0.0
x(s8,t2) = 0.0
x(s9,t2) = 0.0
x(s10,t2) = 0.0
x(s11,t2) = 0.0
x(s12,t2) = 2.0367
x(s13,t2) = 1.9633
x(s14,t2) = 3.0
x(s15,t2) = 0.9615
x(s16,t2) = 3.0

Property 1 values:

q1[t1] = 0.4354
q1[t2] = 2.3969

Property 2 values:

q2[t1] = 30.0
q2[t2] = 32.0


We can remove the pool nodes by removing the constraints (5), (6) and (9).

The increase in the objective value is caused by not being limited by the flow capacities of the pools, i.e. constraint (6). 